In [1]:
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled
from langchain_text_splitters.character import RecursiveCharacterTextSplitter
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai.embeddings import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores.faiss import FAISS
from langchain_core.prompts import PromptTemplate
from dotenv import load_dotenv
load_dotenv()

True

## Step 1a - Indexing (Document Ingestion)

In [2]:
video_id = "f8dhP521DHI" # only the ID, not full URL
try:
    # If you don’t care which language, this returns the “best” one
    ytt_api = YouTubeTranscriptApi()
    transcript_list = ytt_api.fetch(video_id)
    print(transcript_list)
    # Flatten it to plain text
    transcript = " ".join(chunk.text for chunk in transcript_list)
    print(transcript)

except TranscriptsDisabled:
    print("No captions available for this video.")
except Exception as e:
  print('Other error', e)

FetchedTranscript(snippets=[FetchedTranscriptSnippet(text='So as a final example of divide and conquer, we\xa0\nwill look at a problem called quick select.\n\xa0', start=9.92, duration=6.0), FetchedTranscriptSnippet(text='So selection is the problem of\xa0\nfinding a specific position in\xa0\xa0', start=17.28, duration=4.72), FetchedTranscriptSnippet(text='a list. So in particular we want to find the\xa0\nkth largest value in a sequence of length n.\xa0\xa0', start=22.0, duration=5.76), FetchedTranscriptSnippet(text='So of course one way to do this is to sort the\xa0\nlist in descending order, and then the largest\xa0\xa0', start=28.8, duration=6.16), FetchedTranscriptSnippet(text='value is the first one, the second largest value\xa0\nis the second one and so on, so by just looking at\xa0\xa0', start=34.96, duration=4.48), FetchedTranscriptSnippet(text='position k in the sorted list in descending\xa0\norder we will get the kth largest value.\n\xa0', start=39.44, duration=4.4), FetchedT

In [10]:
transcript

'So as a final example of divide and conquer, we\xa0\nwill look at a problem called quick select.\n\xa0 So selection is the problem of\xa0\nfinding a specific position in\xa0\xa0 a list. So in particular we want to find the\xa0\nkth largest value in a sequence of length n.\xa0\xa0 So of course one way to do this is to sort the\xa0\nlist in descending order, and then the largest\xa0\xa0 value is the first one, the second largest value\xa0\nis the second one and so on, so by just looking at\xa0\xa0 position k in the sorted list in descending\xa0\norder we will get the kth largest value.\n\xa0 So since we know we can sort in n log n,\xa0\nthis would mean that the kth largest value,\xa0\xa0 the selection problem of finding the\xa0\nkth largest value also takes n log n.\xa0\xa0 But if we look at some special cases it perhaps\xa0\nsuggests that we could do better. For instance,\xa0\xa0 if we want k equal to 1 then we are just\xa0\nlooking for the largest value in the list\xa0\xa0 and that is

In [68]:
# for snippet in transcript_list:
#     # print(snippet["text"])
for chunk in transcript_list:
    print(chunk.text)
# print(transcript)

So as a final example of divide and conquer, we 
will look at a problem called quick select.
 
So selection is the problem of 
finding a specific position in  
a list. So in particular we want to find the 
kth largest value in a sequence of length n.  
So of course one way to do this is to sort the 
list in descending order, and then the largest  
value is the first one, the second largest value 
is the second one and so on, so by just looking at  
position k in the sorted list in descending 
order we will get the kth largest value.
 
So since we know we can sort in n log n, 
this would mean that the kth largest value,  
the selection problem of finding the 
kth largest value also takes n log n.  
But if we look at some special cases it perhaps 
suggests that we could do better. For instance,  
if we want k equal to 1 then we are just 
looking for the largest value in the list  
and that is just the maximum. And we know 
that we can compute the maximum in a list  
in a single scan so i

In [69]:
print(type(transcript))

<class 'str'>


## Step 1b - Indexing (Text Splitting)

In [11]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap= 200,
)
chunks = splitter.create_documents([transcript])

In [22]:
print(chunks[0].page_content)

So as a final example of divide and conquer, we 
will look at a problem called quick select.
  So selection is the problem of 
finding a specific position in   a list. So in particular we want to find the 
kth largest value in a sequence of length n.   So of course one way to do this is to sort the 
list in descending order, and then the largest   value is the first one, the second largest value 
is the second one and so on, so by just looking at   position k in the sorted list in descending 
order we will get the kth largest value.
  So since we know we can sort in n log n, 
this would mean that the kth largest value,   the selection problem of finding the 
kth largest value also takes n log n.   But if we look at some special cases it perhaps 
suggests that we could do better. For instance,   if we want k equal to 1 then we are just 
looking for the largest value in the list   and that is just the maximum. And we know


In [72]:
print(chunks[10].page_content)

1, where m is now for me, this lower length.
  So what we have done is we basically taken quick 
sort, and we have taken the partition algorithm   exactly as in quick sort, and then after 
partitioning, instead of sorting the two halves   we are suitably choosing which part of the 
partition to select. So either we select in   the lower part or in the upper part or 
we just directly return the pivot.
  So if you want to analyze this, then the 
recurrence is very similar to quick sort.   The only difference is we are not going to look 
at both halves. So we are going to look at let,   let m be the length of the lower part. So we are 
either going to look at the lower part or we are   going to look at the upper part, and in the 
worst case we have to go with a larger one.
  So instead of saying T of m plus T of n minus 
m plus 1, which is what we would do if we had   quick sort because we would have to solve both


## Step 1c & 1d - Indexing (Embedding Generation and Storing in Vector Store)

In [23]:
embedding_model = GoogleGenerativeAIEmbeddings(model = "models/gemini-embedding-001")
vector_store = FAISS.from_documents(
    embedding=embedding_model,
    documents=chunks
)

In [24]:
vector_store.index_to_docstore_id

{0: '524092a9-30cd-4c2f-bd09-a84f53b47728',
 1: '9f9abd4b-9160-4217-a366-af1299331911',
 2: '705fc287-9c97-41e3-b7bb-1434a5464104',
 3: '959269b3-c55b-41d4-be0e-2d555dd9e0ed',
 4: 'a7fa5670-975e-4e8e-9c0d-8e6aa6581f75',
 5: '19319dbf-a727-42a5-9f69-e7f4a1e04f97',
 6: '87427d31-854d-401b-8418-829677c70acd',
 7: '6ca10745-3cb0-4209-998c-9c1fa6cc54d9',
 8: '41cd97c7-e46b-42d8-a7fa-81ef24627149',
 9: '01a8c310-5e90-47a8-bbbd-ad735e96fe1d',
 10: 'b7823af4-359b-4911-bca6-163670173c6d',
 11: '945a7f2b-4cfc-4547-9bf7-b0755343ca82',
 12: '773dd88d-c81b-4173-8c93-362e37f8fe07',
 13: '0e004da7-fabc-44be-9ac1-a48b261cebd4',
 14: '3f24653d-890b-4b61-9ec6-45ca0c5d7353',
 15: '66e4aba6-4801-4647-819a-a356b2a5f5f7',
 16: 'eb00c6fc-f5f3-445f-917c-24a47e22ff28',
 17: '4a4e227b-9d49-4da4-8d5a-3343033278e9',
 18: 'd810ded5-aace-4fe2-b93e-27493192e03b',
 19: '95afa6ff-b5da-424e-b143-c9c3ed347a1d',
 20: '257d2f25-a7fa-4492-aae2-db1b60c6e13b',
 21: 'ee36eb07-72c2-42be-a140-ed0c001915cf',
 22: 'e88c1f8d-f91b-

In [30]:
vector_store.get_by_ids(["1f130fb3-c9fa-47b1-a480-36d52d3feff8"])[0].page_content

'order for our quick select or quick sort recursion\xa0\xa0 to give us an n log n or order n thing.\nSo, so let us see why the median of medians\xa0\xa0 that we are constructing is a good value.\xa0\nSo if we visualize the blocks, so this is\xa0\xa0 my block 1. So these are my first five elements.\xa0\nSo each column in this is, so this is my entire,\xa0\xa0 from left to right, is my entire list, and I\xa0\nhave been grouping it in groups of five.\n\xa0 Now of course, each of these five is not sorted\xa0\nbut let us pretend that it is sorted. So let us\xa0\xa0 visualize it so that each of these blocks of\xa0\nfive is internally sorted like we did here. So\xa0\xa0 we are looking at this value\xa0\nthis x after it is sorted.\n\xa0 So we keep this. So we are not actually doing\xa0\nthis in the original list. We are just doing it\xa0\xa0 block by block but let us see imagine that we\xa0\nactually collected all the sorted blocks. Then\xa0\xa0 where would the median of each block lie? It wil

## Step 2 - Retrieval

In [31]:
retriever = vector_store.as_retriever(search_type = 'mmr', search_kwargs={"k": 3, "lambda_mult": 0.5})

In [32]:
query = "what is divide and conqur"
retriever.invoke(query)

[Document(id='524092a9-30cd-4c2f-bd09-a84f53b47728', metadata={}, page_content='So as a final example of divide and conquer, we\xa0\nwill look at a problem called quick select.\n\xa0 So selection is the problem of\xa0\nfinding a specific position in\xa0\xa0 a list. So in particular we want to find the\xa0\nkth largest value in a sequence of length n.\xa0\xa0 So of course one way to do this is to sort the\xa0\nlist in descending order, and then the largest\xa0\xa0 value is the first one, the second largest value\xa0\nis the second one and so on, so by just looking at\xa0\xa0 position k in the sorted list in descending\xa0\norder we will get the kth largest value.\n\xa0 So since we know we can sort in n log n,\xa0\nthis would mean that the kth largest value,\xa0\xa0 the selection problem of finding the\xa0\nkth largest value also takes n log n.\xa0\xa0 But if we look at some special cases it perhaps\xa0\nsuggests that we could do better. For instance,\xa0\xa0 if we want k equal to 1 then

## Step 3 - Augmentation

In [ ]:
# context = 
# query = 
prompt = PromptTemplate(
    template = """You are a helpful teaching assistent
                answer the question asked by student based on the given and If you don't know the answer just say "I don't know"
                question: \n {query} \n
                context \n {context} \n
                """,
    # input_variables = ['context', 'query']
)
print(prompt)

input_variables=['context', 'query'] input_types={} partial_variables={} template='You are a helpful teaching assistent\n                answer the question asked by student based on the given and If you don\'t know the answer just say "I don\'t know"\n                question: \n {query} \n\n                context \n {context} \n\n                '


In [34]:
query = "what is divide and conqur"
context_doc = retriever.invoke(query)
context_text = " ".join(content.page_content for content in context_doc)
context_text

'So as a final example of divide and conquer, we\xa0\nwill look at a problem called quick select.\n\xa0 So selection is the problem of\xa0\nfinding a specific position in\xa0\xa0 a list. So in particular we want to find the\xa0\nkth largest value in a sequence of length n.\xa0\xa0 So of course one way to do this is to sort the\xa0\nlist in descending order, and then the largest\xa0\xa0 value is the first one, the second largest value\xa0\nis the second one and so on, so by just looking at\xa0\xa0 position k in the sorted list in descending\xa0\norder we will get the kth largest value.\n\xa0 So since we know we can sort in n log n,\xa0\nthis would mean that the kth largest value,\xa0\xa0 the selection problem of finding the\xa0\nkth largest value also takes n log n.\xa0\xa0 But if we look at some special cases it perhaps\xa0\nsuggests that we could do better. For instance,\xa0\xa0 if we want k equal to 1 then we are just\xa0\nlooking for the largest value in the list\xa0\xa0 and that is

In [35]:
final_prompt = prompt.invoke({"query":query, "context": context_text })

In [38]:
print(final_prompt)

text='You are a helpful teaching assistent\n                answer the question asked by student based on the given and If you don\'t know the answer just say "I don\'t know"\n                question: \n what is divide and conqur \n\n                context \n So as a final example of divide and conquer, we\xa0\nwill look at a problem called quick select.\n\xa0 So selection is the problem of\xa0\nfinding a specific position in\xa0\xa0 a list. So in particular we want to find the\xa0\nkth largest value in a sequence of length n.\xa0\xa0 So of course one way to do this is to sort the\xa0\nlist in descending order, and then the largest\xa0\xa0 value is the first one, the second largest value\xa0\nis the second one and so on, so by just looking at\xa0\xa0 position k in the sorted list in descending\xa0\norder we will get the kth largest value.\n\xa0 So since we know we can sort in n log n,\xa0\nthis would mean that the kth largest value,\xa0\xa0 the selection problem of finding the\xa0\nk

## Step 4 - Generation

In [39]:
model = ChatGoogleGenerativeAI(model='models/gemini-2.0-flash')
model.invoke(final_prompt)

AIMessage(content='Based on the context you provided, divide and conquer is a strategy used in algorithms, exemplified here by the "quick select" algorithm. The general idea seems to be to break a problem into smaller subproblems, solve those subproblems (often recursively), and then combine the solutions to solve the original problem.\n\nIn the context of quick select:\n\n*   The problem is to find the kth largest element in a list.\n*   The algorithm aims to find a "good pivot" that will split the list into two partitions.\n*   The goal is to guarantee that at least a fixed fraction of elements will be pushed into one of the two partitions, avoiding worst-case scenarios.\n*   A "median of medians" approach is mentioned as a way to find a good pivot. This involves dividing the list into blocks, finding the median of each block, and then recursively finding the median of those medians.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': [

## Building a Chain

In [40]:
from langchain_core.runnables import RunnableParallel, RunnableLambda,  RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

In [47]:
def myfunc(doc):
    return "\n\n".join(content.page_content for content in doc)

retriever_chain = retriever | RunnableLambda(myfunc)

In [ ]:
augmentation_chain = RunnableParallel({
    "query": RunnablePassthrough(),
    "context": retriever_chain
})

model_chain = augmentation_chain | prompt | model | StrOutputParser()

In [ ]:
print(augmentation_chain.invoke("what is the divide and conqer method??"))2

{'query': 'what is the divide and conqer method??', 'context': 'So as a final example of divide and conquer, we\xa0\nwill look at a problem called quick select.\n\xa0 So selection is the problem of\xa0\nfinding a specific position in\xa0\xa0 a list. So in particular we want to find the\xa0\nkth largest value in a sequence of length n.\xa0\xa0 So of course one way to do this is to sort the\xa0\nlist in descending order, and then the largest\xa0\xa0 value is the first one, the second largest value\xa0\nis the second one and so on, so by just looking at\xa0\xa0 position k in the sorted list in descending\xa0\norder we will get the kth largest value.\n\xa0 So since we know we can sort in n log n,\xa0\nthis would mean that the kth largest value,\xa0\xa0 the selection problem of finding the\xa0\nkth largest value also takes n log n.\xa0\xa0 But if we look at some special cases it perhaps\xa0\nsuggests that we could do better. For instance,\xa0\xa0 if we want k equal to 1 then we are just\xa0

In [57]:
result = model_chain.invoke("summarize the video??")

In [60]:
result

AIMessage(content='The video discusses the "median of medians" algorithm, a technique developed by Manuel Blum, Robert Floyd, Vaughn Pratt, Ron Rivest, and Robert Tarjan to improve the speed of quick select and quick sort. The algorithm involves dividing the data into blocks, finding the median of each block, and then finding the median of those medians. The video explains how this "median of medians" (the blue element) is used to determine how many elements in the original data are smaller than it (the brown colored elements). The video also highlights that in the worst-case scenario, quick select can still have a time complexity of n-squared, similar to a brute-force approach.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--4e3c9fa4-7f8d-4192-a047-ba62ae821a27-0', usage_metadata={'input_tokens': 612, '

In [66]:
StrOutputParser().invoke(result)

'The video discusses the "median of medians" algorithm, a technique developed by Manuel Blum, Robert Floyd, Vaughn Pratt, Ron Rivest, and Robert Tarjan to improve the speed of quick select and quick sort. The algorithm involves dividing the data into blocks, finding the median of each block, and then finding the median of those medians. The video explains how this "median of medians" (the blue element) is used to determine how many elements in the original data are smaller than it (the brown colored elements). The video also highlights that in the worst-case scenario, quick select can still have a time complexity of n-squared, similar to a brute-force approach.'